In [6]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import SystemMessagePromptTemplate, PromptTemplate
from vector_store import youtube_url_to_documents
from prompt import format_context_for_llm


In [7]:
load_dotenv()

True

In [8]:
llm = ChatOpenAI(
    model="gpt-4o",
)

In [9]:
system_prompt = SystemMessage("The user watches a video, and after sometime he wants to comeback to a particular point in the video discussing about some topic. You are a YouTube assistant that helps users find the timestamp in a video. When given a transcript and its timestamps, provide the timestamp(s) for the requested topic.")

In [10]:
chat_history = [system_prompt]

In [40]:
from typing import List
from pydantic import Field, field_validator
import re

class TimeStamp(BaseModel):
    timestamp: List[str] = Field(description='This has a list of timestamps in MM:SS format like ["20:53", "20:58"]')

    @field_validator("timestamp")
    def validate_timestamp(cls, v):
        for ts in v:
            if not re.match(r"^\d{1,2}:\d{2}$", ts):
                raise ValueError("Timestamp must be in MM:SS format")
            minutes, seconds = map(int, ts.split(":"))
            if not (0 <= minutes and 0 <= seconds < 60):
                raise ValueError("Minutes must be >= 0 and seconds must be between 0 and 59")
        return v


In [41]:
parser = PydanticOutputParser(pydantic_object=TimeStamp)

# Prompt template

In [46]:
user_input = "common loss function"

In [47]:
user_url_input = "https://www.youtube.com/watch?v=FLkUOkeMd5M"

In [48]:
user_url_input

'https://www.youtube.com/watch?v=FLkUOkeMd5M'

In [16]:
vector_store = youtube_url_to_documents(user_url_input)
context = vector_store.similarity_search(
    query=user_input,
    k=5
)

formatted_context = format_context_for_llm(context)

In [17]:
context

[Document(metadata={'timestamp': '20:58', 'https://www.youtube.com/watch?v=zxQyTK8quyY': 'zxQyTK8quyY'}, page_content='parallel Computing and run fast'),
 Document(metadata={'timestamp': '20:58', 'https://www.youtube.com/watch?v=zxQyTK8quyY': 'zxQyTK8quyY'}, page_content='parallel Computing and run fast'),
 Document(metadata={'timestamp': '20:58', 'url': 'zxQyTK8quyY'}, page_content='parallel Computing and run fast'),
 Document(metadata={'timestamp': '20:53', 'https://www.youtube.com/watch?v=zxQyTK8quyY': 'zxQyTK8quyY'}, page_content='computation at the same time'),
 Document(metadata={'url': 'zxQyTK8quyY', 'timestamp': '20:53'}, page_content='computation at the same time')]

In [42]:
template = PromptTemplate(template="{system_prompt}\n use the followingt context and timestamps\n{context}\n Find the timestamps for the following user request\n{input}\n {format_instruction}",
               input_variables=["system_prompt" ,"context", "input"],
               partial_variables={'format_instruction':parser.get_format_instructions()})

In [43]:
prompt = template.invoke(
    {
        "system_prompt":system_prompt.content,
        "context": formatted_context,
        "input": user_input
    }
)

In [44]:
prompt

StringPromptValue(text='The user watches a video, and after sometime he wants to comeback to a particular point in the video discussing about some topic. You are a YouTube assistant that helps users find the timestamp in a video. When given a transcript and its timestamps, provide the timestamp(s) for the requested topic.\n use the followingt context and timestamps\nTimestamp: 20:58\nContent: parallel Computing and run fast\n\nTimestamp: 20:58\nContent: parallel Computing and run fast\n\nTimestamp: 20:58\nContent: parallel Computing and run fast\n\nTimestamp: 20:53\nContent: computation at the same time\n\nTimestamp: 20:53\nContent: computation at the same time\n\n Find the timestamps for the following user request\nparallel computation?\n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "re

In [32]:
response = llm.invoke(prompt)


In [33]:
response

AIMessage(content='```json\n{"timestamp": ["20:58", "20:53"]}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 330, 'total_tokens': 347, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BcBHhbbCc6qlPtGrvsLYstWhGsrfm', 'finish_reason': 'stop', 'logprobs': None}, id='run--29d159d1-d318-412a-bae0-88e76546166f-0', usage_metadata={'input_tokens': 330, 'output_tokens': 17, 'total_tokens': 347, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [45]:
parser.invoke(response)

TimeStamp(timestamp=['20:58', '20:53'])

# Chains

In [49]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

# Define a lambda to retrieve context from the vector store
def retrieve_context(inputs):
    vector_store = youtube_url_to_documents(inputs["url"])
    context = vector_store.similarity_search(
        query=inputs["query"],
        k=5
    )
    return {"context": format_context_for_llm(context)}

# Define a lambda to format the prompt
def build_prompt(inputs):
    return template.invoke({
        "system_prompt": system_prompt.content,
        "context": inputs["context"],
        "input": inputs["query"]
    })

# Parallel chain: retrieves context and builds prompt in parallel, then sends to LLM
parallel_chain = (
    RunnableParallel(
        context=RunnableLambda(retrieve_context),
        query=lambda x: x["query"]
    )
    | RunnableLambda(lambda inputs: {
        "prompt": build_prompt({"context": inputs["context"]["context"], "query": inputs["query"]})
    })
    | RunnableLambda(lambda inputs: llm.invoke(inputs["prompt"]))
    | parser
)

# Example usage:
inputs = {
    "url": user_url_input,
    "query": user_input
}
result = parallel_chain.invoke(inputs)
print(result)

timestamp=['17:02', '18:27', '18:50', '19:45', '19:01']


In [51]:
result.timestamp

['17:02', '18:27', '18:50', '19:45', '19:01']